In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
import lightning as L
from torch.utils.data import TensorDataset, DataLoader

In [2]:
class LightningLSTM(L.LightningModule):
  def __init__(self):
    super().__init__()

    self.lstm = nn.LSTM(input_size=1, hidden_size=1)
  
  def forward(self, input):
    #Transpose the input tensor from 1 x Input_size to Input_size x 1
    # input.view(number_of_rows, number_of_columns)
    input_transpose = input.view(len(input), 1)

    #lstm contains short-term-memory values for each LSTM unit (would be size of input)
    lstm_out, temp = self.lstm(input_transpose)
    prediction = lstm_out[-1]
    return prediction

  def configure_optimizers(self):
    return Adam(self.parameters())
  
  def training_step(self, batch):
    input_i, label_i = batch
    output_i = self.forward(input_i[0]) # Since input_i is a 2d tensor, access the 1d list by indexing 0
    loss = (output_i - label_i)**2

    self.log("train_loss", loss)

    if label_i == 0:
      self.log("out_0", output_i)
    else:
      self.log("out_1", output_i)

    return loss
  
model = LightningLSTM()

In [3]:
inputs = torch.tensor([[0., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]])
labels = torch.tensor([0., 1.])

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [4]:
trainer = L.Trainer(max_epochs=2000)
trainer.fit(model, train_dataloaders=dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type | Params
------------------------------
0 | lstm | LSTM | 16    
------------------------------
16        Trainable params
0         Non-trainable params
16        Total params
0.000     Total estimated model params size (MB)
d:\python3115\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
d:\python3115\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 1999: 100%|██████████| 2/2 [00:00<00:00, 17.59it/s, v_num=0] 

`Trainer.fit` stopped: `max_epochs=2000` reached.


Epoch 1999: 100%|██████████| 2/2 [00:00<00:00, 15.53it/s, v_num=0]


In [5]:
company_a_pred = model(torch.tensor([0., 0.5, 0.25, 1.]))
company_b_pred = model(torch.tensor([1., 0.5, 0.25, 1.]))
print(f"COMPANY A: observed = 0, predicted = {company_a_pred.detach().item():0.4f}")
print(f"COMPANY B: observed = 1, predicted = {company_b_pred.detach().item():0.4f}")

COMPANY A: observed = 0, predicted = 0.0081
COMPANY B: observed = 1, predicted = 0.8116


In [6]:
path_to_best_checkpoint = trainer.checkpoint_callback.best_model_path
trainer = L.Trainer(max_epochs=4000)
trainer.fit(model, train_dataloaders=dataloader, ckpt_path=path_to_best_checkpoint)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at d:\githubrepo\LSTM-practice\LSTM_pytorch\lightning_logs\version_0\checkpoints\epoch=1999-step=4000.ckpt
d:\python3115\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:360: The dirpath has changed from 'd:\\githubrepo\\LSTM-practice\\LSTM_pytorch\\lightning_logs\\version_0\\checkpoints' to 'd:\\githubrepo\\LSTM-practice\\LSTM_pytorch\\lightning_logs\\version_1\\checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type | Params
------------------------------
0 | lstm | LSTM | 16    
------------------------------
16        Trainable params
0         Non-trainable params
16        Total params
0.000    

Epoch 3999: 100%|██████████| 2/2 [00:00<00:00, 22.16it/s, v_num=1] 

`Trainer.fit` stopped: `max_epochs=4000` reached.


Epoch 3999: 100%|██████████| 2/2 [00:00<00:00, 19.38it/s, v_num=1]


In [7]:
company_a_pred = model(torch.tensor([0., 0.5, 0.25, 1.]))
company_b_pred = model(torch.tensor([1., 0.5, 0.25, 1.]))
print(f"COMPANY A: observed = 0, predicted = {company_a_pred.detach().item():0.4f}")
print(f"COMPANY B: observed = 1, predicted = {company_b_pred.detach().item():0.4f}")

COMPANY A: observed = 0, predicted = 0.0001
COMPANY B: observed = 1, predicted = 0.9632
